# Amplifierd API - Collection Management

This notebook demonstrates collection discovery and management operations.

## Overview

Collections in amplifier provide:
- Reusable profiles
- Pre-configured agents
- Module packages (providers, tools, hooks, orchestrators)
- Shared configurations

This notebook covers both **read operations** (Phase 1) and **write operations** (Phase 2).

In [ ]:
import json

import requests

BASE_URL = "http://127.0.0.1:8420"
API_BASE = f"{BASE_URL}/api/v1"


def print_response(response: requests.Response, title: str = "") -> None:
    if title:
        print(f"\n{'=' * 60}")
        print(f"{title}")
        print(f"{'=' * 60}")
    print(f"Status: {response.status_code} {response.reason}")
    if response.content:
        try:
            data = response.json()
            print(json.dumps(data, indent=2))
            return data
        except json.JSONDecodeError:
            print(response.text)
            return None
    return None


print("✓ Setup complete")

## Phase 1: Read Operations

### List All Collections

Get all available collections:

In [ ]:
response = requests.get(f"{API_BASE}/collections/")
collections = print_response(response, "LIST COLLECTIONS")

if collections:
    print(f"\n✓ Found {len(collections)} collection(s)")
    for collection in collections:
        print(f"  - {collection['identifier']} ({collection['type']})")
        print(f"    Source: {collection['source']}")

### Get Collection Details

Retrieve detailed information about a specific collection:

In [ ]:
# Get details for a collection (change identifier as needed)
if collections:
    collection_id = collections[0]["identifier"]

    response = requests.get(f"{API_BASE}/collections/{collection_id}")
    details = print_response(response, f"GET COLLECTION: {collection_id}")

    if details:
        print(f"\n✓ Collection: {details['identifier']}")
        print(f"  Type: {details['type']}")
        print(f"  Profiles: {len(details.get('profiles', []))}")
        print(f"  Agents: {len(details.get('agents', []))}")

        modules = details.get("modules", {})
        print("  Modules:")
        print(f"    - Providers: {len(modules.get('providers', []))}")
        print(f"    - Tools: {len(modules.get('tools', []))}")
        print(f"    - Hooks: {len(modules.get('hooks', []))}")
        print(f"    - Orchestrators: {len(modules.get('orchestrators', []))}")
else:
    print("No collections available. Try mounting one first.")

### Explore Collection Resources

List all profiles and modules in a collection:

In [ ]:
if collections and details:
    print("Profiles in collection:")
    for profile_path in details.get("profiles", []):
        print(f"  - {profile_path}")

    print("\nProvider modules:")
    for provider in details.get("modules", {}).get("providers", []):
        print(f"  - {provider}")

    print("\nTool modules:")
    for tool in details.get("modules", {}).get("tools", []):
        print(f"  - {tool}")

## Phase 2: Write Operations

### Mount a Collection

Add a new collection to your amplifier configuration:

**Note**: This modifies your local configuration. You can use a git URL or local path.

In [ ]:
# Example: Mount a collection from git
mount_data = {"identifier": "example-collection", "source": "https://github.com/example/amplifier-collection.git"}

# WARNING: This will attempt to clone the repository
# Comment out if you don't want to modify your configuration

# response = requests.post(
#     f"{API_BASE}/collections/",
#     json=mount_data
# )
# result = print_response(response, "MOUNT COLLECTION")
#
# if response.status_code == 201:
#     print(f"\n✓ Mounted: {result['identifier']}")
#     print(f"  Source: {result['source']}")
# elif response.status_code == 409:
#     print(f"\n✗ Collection already mounted")
# elif response.status_code == 400:
#     print(f"\n✗ Invalid source or failed to clone")

print("ℹ Mount example commented out to prevent accidental changes")
print("  Uncomment the code above to actually mount a collection")

### Mount from Local Path

You can also mount collections from local filesystem:

In [ ]:
# Example: Mount from local path
local_mount_data = {"identifier": "local-collection", "source": "/path/to/local/collection"}

# Commented for safety
# response = requests.post(
#     f"{API_BASE}/collections/",
#     json=local_mount_data
# )
# print_response(response, "MOUNT LOCAL COLLECTION")

print("ℹ Local mount example shown (commented for safety)")

### Unmount a Collection

Remove a collection from your configuration:

In [ ]:
# Unmount a collection
collection_to_unmount = "example-collection"

# WARNING: This will remove the collection from your configuration
# Comment out if you don't want to modify your configuration

# response = requests.delete(f"{API_BASE}/collections/{collection_to_unmount}")
# result = print_response(response, f"UNMOUNT: {collection_to_unmount}")
#
# if response.ok:
#     print(f"\n✓ Unmounted successfully")
# elif response.status_code == 404:
#     print(f"\n✗ Collection '{collection_to_unmount}' not found")

print("ℹ Unmount example commented out to prevent accidental changes")

### Error Handling: Already Mounted

Attempting to mount a collection that's already mounted:

In [ ]:
# This will return 409 Conflict if collection already exists
# Uncomment to test:

# duplicate_mount = {
#     "identifier": "existing-collection",  # Use an existing collection ID
#     "source": "https://github.com/example/collection.git"
# }
#
# response = requests.post(
#     f"{API_BASE}/collections/",
#     json=duplicate_mount
# )
# print_response(response, "MOUNT DUPLICATE")
#
# if response.status_code == 409:
#     print("\n✓ Correctly returns 409 for already-mounted collection")

print("ℹ Duplicate mount example shown (commented for safety)")

## Complete Collection Workflow

Demonstrate discovery and exploration:

In [ ]:
def collection_discovery_workflow():
    """Explore available collections and their resources."""

    # 1. List all collections
    print("1. Listing collections...")
    response = requests.get(f"{API_BASE}/collections/")
    if not response.ok:
        print("✗ Failed to list collections")
        return

    collections = response.json()
    print(f"✓ Found {len(collections)} collection(s)")

    # 2. Explore each collection
    for collection in collections:
        identifier = collection["identifier"]
        print(f"\n2. Exploring '{identifier}'...")

        response = requests.get(f"{API_BASE}/collections/{identifier}")
        if response.ok:
            details = response.json()
            modules = details.get("modules", {})
            total_modules = (
                len(modules.get("providers", []))
                + len(modules.get("tools", []))
                + len(modules.get("hooks", []))
                + len(modules.get("orchestrators", []))
            )
            print("✓ Collection has:")
            print(f"  - {len(details.get('profiles', []))} profiles")
            print(f"  - {total_modules} modules")
            print(f"  - {len(details.get('agents', []))} agents")

    print("\n✓ Discovery complete")


collection_discovery_workflow()

## Summary

### Phase 1 Operations (Read-Only)
- ✓ List all available collections
- ✓ Get collection details and resources
- ✓ Explore profiles, agents, and modules in collections

### Phase 2 Operations (Write)
- ✓ Mount collections (from git or local path)
- ✓ Unmount collections
- ✓ Error handling (already mounted, not found, invalid source)

## API Endpoints Reference

| Method | Endpoint | Description | Phase |
|--------|----------|-------------|-------|
| GET | `/api/v1/collections/` | List all collections | 1 |
| GET | `/api/v1/collections/{id}` | Get collection details | 1 |
| POST | `/api/v1/collections/` | Mount collection | 2 |
| DELETE | `/api/v1/collections/{id}` | Unmount collection | 2 |

## Important Notes

### Collection Identifiers with Slashes

Collection identifiers may contain slashes (e.g., `github.com/org/repo`). The API handles these correctly:

```python
# Works correctly with slashes
collection_id = "github.com/org/repo"
response = requests.get(f"{API_BASE}/collections/{collection_id}")
```

### Configuration Files Modified

Write operations modify:
- `~/.amplifier/collections.lock` - Collection mount registry
- Downloaded collections are stored in `~/.amplifier/collections/`

## Next Steps

Continue to:
- **05-module-management.ipynb** - Module discovery and configuration